In [1]:
import os
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pickle

In [2]:
PATH = os.getcwd()

In [3]:
LOG_DIR = PATH + '/project-tensorboard/log-1/'

In [4]:
with open('raw_df.pkl', 'rb') as f:
    audio_df = pickle.load(f)

In [64]:
features = audio_df.iloc[:, 0: -2]
labels = audio_df.iloc[:, -2]

In [76]:
labels = labels.replace({0:'AC',1:'Car Horn',2:'Children Playing',3:'Dog',4:'Drill',
                         5:'Engine Idle',6:'Gun Shot',7:'Jackhammer',8:'Siren',9:'Street Music'})

In [75]:
# fold1 = audio_df[audio_df.fold == 'fold1']
# features = fold1.iloc[:, 0: -2]
# labels = fold1.iloc[:, -2]

8732

In [77]:
scaler = StandardScaler()
features = scaler.fit_transform(features)
features = pd.DataFrame(features)

In [78]:
# pca = PCA(n_components=50,
#          random_state = 123,
#          svd_solver = 'auto'
#          )

In [79]:
# df_pca = pd.DataFrame(pca.fit_transform(features))
# df_pca = df_pca.values

In [80]:
# tf_data = tf.Variable(df_pca)

In [81]:
tf_data = tf.Variable(features.values)

In [82]:
labels.to_csv('df_labels.tsv', sep='\t', encoding='utf8', header=True)

In [83]:
metadata = os.path.join(LOG_DIR, 'df_labels.tsv')

In [84]:
with tf.Session() as sess:
    saver = tf.train.Saver([tf_data])
    sess.run(tf_data.initializer)
    saver.save(sess, os.path.join(LOG_DIR, 'tf_data.ckpt'))
    config = projector.ProjectorConfig()
    
    embedding = config.embeddings.add()
    embedding.tensor_name = tf_data.name
    
    embedding.metadata_path = metadata
    
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)

Run with: tensorboard --logdir=/home/ubuntu/Notebooks/project-tensorboard/log-1/ --port=6006